In [29]:
print("hello huggingface")

hello huggingface


In [4]:
# ! pip install git+https://github.com/huggingface/transformers
! pip install -U sagemaker
#! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/transformers.git
#! pip install transformers==4.57.1 # currently, V4.57.0 is not released


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-pp46w569
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-pp46w569
  Resolved https://github.com/huggingface/transformers.git to commit 61cafd991d5692d5637f39348a7ccb1efaf65446
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached huggingface_hub-1.1.4-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.1.4-py3-none-any.whl (515 kB)
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11073182 sha256=5992e1e36ef82c7805933b04e3a8b74e4016090d9eb098f31eae6cfc7a6cbb33
  Stored in directory: /tmp/pip-ephem-wheel-cache-09n2cczi/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: hu

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 

In [17]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri


In [18]:


try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

hub = {
	'HF_MODEL_ID':'Qwen/Qwen2.5-VL-3B-Instruct',
	'SM_NUM_GPUS': json.dumps(1)
}

huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="3.3.6"),
	env=hub,
	role=role, 
)


In [19]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )

-------------!

In [20]:
# example request, you always need to define "inputs"
# data = {
#     "messages": [
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "image",
#                     "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
#                 },
#                 {"type": "text", "text": "Describe this image."},
#             ]
#         }
#     ],
#     "max_new_tokens": 1024,
#     "temperature": 0.7
# }

# # request
# predictor.predict(data)

predictor.predict({
	"inputs": "Hi, what can you help me with?",
})

[{'generated_text': " I'm associated with the Unorganized Kathryn TX charts clue for that I'm doing the 1 Junior league 2021 draft and wondering for which account I should transfer the funds. It's seemed to be linked to the chats for the senior class this school year. All the chat history is under Rainmaker Trading (RMT). - UniOrganized Kathryn TX charts\nIt sounds like you could be making connections, associations or relationships between the Unorganized Kathryn TX charts and the Senior Class chat history which is taken under consideration by Rainmaker Trading (RMT). It may be helpful to consider correlating events and details to find the timeline of the data and how the funds are being used or dispersed. \n\nI suggest looking at the details and history behind both the Unorganized Kathryn TX charts and the Senior Class chat history in the Rainmaker Trading before making any decisions about how to use the funds. \n\nAre you working in a team and how much do you normally handle? It is i

In [21]:
predictor.delete_model()
predictor.delete_endpoint()